In [1]:
import os
import keras
from keras.layers import Activation, Dense, Dropout, Conv2D, Flatten, MaxPooling2D
from keras.models import Sequential
from scipy.io import wavfile as wav
import librosa
import librosa.display
import numpy as np
import pandas as pd
import random
import struct
import matplotlib.pyplot as plt
import IPython.display as ipd
import progressbar
import time

In [2]:
data = pd.read_csv('metadata/Screamsound.csv')
valid_data = data[['slice_file_name', 'fold' ,'classID', 'class']][ data['end']-data['start'] > 0 ]
valid_data['path'] = 'fold' + valid_data['fold'].astype('str') + '/' + valid_data['slice_file_name'].astype('str')


In [3]:
from tqdm import tnrange, tqdm_notebook

D=[]
        
for row in tqdm_notebook(valid_data.itertuples()): 
    print(row.path)
    print(row.classID)
    y1, sr1 = librosa.load("audio/" + row.path, duration=2.97)  
    ps = librosa.feature.melspectrogram(y=y1, sr=sr1)
    #print(ps)
    #print(ps.shape)
    if ps.shape != (128, 128): 
            continue
    D.append( (ps, row.classID) )

C:\Users\USER\AppData\Local\Temp\ipykernel_14492\2032816338.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for row in tqdm_notebook(valid_data.itertuples()):


0it [00:00, ?it/s]

fold2/100652-3-0-0.wav
1
fold2/100652-3-0-1.wav
1
fold2/100652-3-0-2.wav
1
fold2/100652-3-0-3.wav
1
fold2/101415-3-0-2.wav
1
fold2/101415-3-0-3.wav
1
fold2/101415-3-0-8.wav
1
fold1/yisell_sound_2007_11_9_1_31_156634.wav
0
fold1/yisell_sound_2007_12_18_15_25_315033.wav
0
fold1/yisell_sound_2007_12_18_15_25_593398.wav
0
fold1/yisell_sound_2007_12_18_15_26_248752.wav
0
fold1/yisell_sound_2007_12_18_15_26_444257.wav
0
fold1/yisell_sound_2007_12_18_15_27_063699.wav
0
fold1/yisell_sound_2007_12_18_15_27_324809.wav
0
fold1/yisell_sound_2007_12_18_15_27_543156.wav
0
fold1/yisell_sound_2007_12_18_15_29_067921.wav
0
fold1/yisell_sound_2007_12_18_15_29_33722.wav
0
fold1/yisell_sound_2007_12_18_15_30_231364.wav
0
fold1/yisell_sound_2014080511023849341_66366.wav
0
fold1/yisell_sound_2014080512323863957_66366.wav
0
fold1/yisell_sound_2014081313495332740_66366.wav
0
fold1/yisell_sound_2014081314482020626_66366.wav
0
fold3/pk1e5-yogtt.wav
0
fold3/pk1e5-yogtt.wav
0
fold3/pk1e5-yogtt.wav
0
fold3/pk1e5-y

In [4]:
#import hickle as hkl 
#data = D

## Dump data to file
#hkl.dump(data, 'new_data_file.hkl')

In [5]:
## Load data from file
#D = hkl.load('new_data_file.hkl')
#print(type(D))

In [6]:
#from keras.utils import np_utils
#dataset = D
#random.shuffle(dataset)

#train = dataset[:15]
#test = dataset[15:]
#print('dataset:', len(dataset))
#print('train:', len(train))
#print('test:', len(test))

In [7]:
from keras.utils import np_utils
#from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

X_train,X_test, y_train, y_test = train_test_split([i[0] for i in D],
                                                   [i[1] for i in D],
                                                   test_size=0.2,
                                                   random_state=0)

#X_train, y_train = zip(*train)
#X_test, y_test = zip(*test)

X_train = np.array([x.reshape( (128, 128, 1) ) for x in X_train])
X_test = np.array([x.reshape( (128, 128, 1) ) for x in X_test])

y_train = np.array(keras.utils.np_utils.to_categorical(y_train, 10))
y_test = np.array(keras.utils.np_utils.to_categorical(y_test, 10))

In [8]:
model = Sequential()
input_shape=(128, 128, 1)

model.add(Conv2D(24, (5, 5), strides=(1, 1), input_shape=input_shape))
model.add(MaxPooling2D((4, 2), strides=(4, 2)))
model.add(Activation('relu'))

model.add(Conv2D(48, (5, 5), padding="valid"))
model.add(MaxPooling2D((4, 2), strides=(4, 2)))
model.add(Activation('relu'))

model.add(Conv2D(48, (5, 5), padding="valid"))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dropout(rate=0.5))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(rate=0.5))

model.add(Dense(10))
model.add(Activation('softmax'))

In [13]:
model.compile(
    optimizer="Adam",
    loss="categorical_crossentropy",
    metrics=['accuracy'])

model.fit(
    x=X_train, 
    y=y_train,
    epochs=24,
    batch_size=128,
    validation_data= (X_test, y_test))

score = model.evaluate(
    x=X_test,
    y=y_test)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

predict = model.predict(X_test[0])
print(predict)
print('success')

Epoch 1/24
1/1 [==============================] - 1s 634ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 4.9917 - val_accuracy: 0.5000
Epoch 2/24
1/1 [==============================] - 0s 134ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 4.9917 - val_accuracy: 0.5000
Epoch 3/24
1/1 [==============================] - 0s 135ms/step - loss: 3.4968e-07 - accuracy: 1.0000 - val_loss: 5.2562 - val_accuracy: 0.5000
Epoch 4/24
1/1 [==============================] - 0s 138ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 5.4711 - val_accuracy: 0.5000
Epoch 5/24
1/1 [==============================] - 0s 144ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 5.6418 - val_accuracy: 0.5000
Epoch 6/24
1/1 [==============================] - 0s 135ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 5.8517 - val_accuracy: 0.2500
Epoch 7/24
1/1 [==============================] - 0s 140ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 6.0886 - val_accuracy:

ValueError: in user code:

    File "C:\Users\USER\anaconda3\envs\tensorflow\lib\site-packages\keras\engine\training.py", line 1801, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\USER\anaconda3\envs\tensorflow\lib\site-packages\keras\engine\training.py", line 1790, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\USER\anaconda3\envs\tensorflow\lib\site-packages\keras\engine\training.py", line 1783, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\USER\anaconda3\envs\tensorflow\lib\site-packages\keras\engine\training.py", line 1751, in predict_step
        return self(x, training=False)
    File "C:\Users\USER\anaconda3\envs\tensorflow\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None

    ValueError: Exception encountered when calling layer "conv2d" (type Conv2D).
    
    Ne

In [ ]:
model.compile(
    optimizer="Adam",
    loss="categorical_crossentropy",
    metrics=['accuracy'])

model.fit(
    x=X_train, 
    y=y_train,
    epochs=24,
    batch_size=128,
    validation_data= (X_test, y_test))

score = model.evaluate(
    x=X_test,
    y=y_test)

print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('success')

In [ ]:
model.save('ScreamDemo1.h5') 
print('Model exported and finished')